In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv("BANKNIFTYFUTALL.csv")
feature_cols_JUN = ['trade', 'event', 'ob', 'spreadreturn','niftyidevent', 'niftyidob','niftyidindex']

#data.dropna(subset=['return800'], axis=0)

data.drop([data.index[457897]],inplace=True)
#data.isnull().sum(axis = 0)

##### Create n samples from the dataset and perform regression on each sample and take the average of the score of each samples

In [9]:
n=10
sco=[]
while(n):
    col=random.randint(2,7)
    row=random.randint(0,457897)
    col_list=[]
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    r_sq = reg.score(X,Y)
    sco.append(r_sq)
    n=n-1
print(sco)
s=0
for i in sco:
    s+=i
print("final average score is ",s/(len(sco)))

[0.0034037210621610248, 0.01886493716081572, 0.003534195162503306, 0.003599351689119623, 0.011826068043760185, 0.0146278638194981, 0.004852107145150165, 0.0028895496954883804, 0.012480587039885749, 0.012414448025891978]
final average score is  0.008849282884427424


###### Considering 100 samples

In [8]:
n=100
sco=[]
while(n):
    col=random.randint(2,7)
    row=random.randint(0,457897)
    col_list=[]
    if(n%2==0):
        feature_cols_JUN.reverse()
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    r_sq = reg.score(X,Y)
    sco.append(r_sq)
    n=n-1
print(sco)
s=0
for i in sco:
    s+=i
print("final average score ",s/(len(sco)))

[0.012321177538974437, 0.01406515900344496, 0.003646596475389896, 0.0030549087889732984, 0.0012811357970994308, 0.000489360109375947, 0.003081427327382702, 0.004525315097405724, 0.0017981835255145295, 0.0012927823136974492, 0.0038969993331289254, 0.0030756266762326234, 0.001155724446404438, 0.00835854579517048, 0.003380062178316101, 0.0031467436116379144, 0.002683649984263514, 0.0019021163336849778, 0.012977013155233474, 0.013228728672017076, 0.004517033882175392, 0.0036682420989616604, 0.00483995318792696, 0.0037785520729137767, 0.0004246053136753858, 0.0075591266656172, 0.009369070350196629, 0.011952940055989236, 0.0004931970658819118, 0.00047236217537238723, 0.0031913708126072304, 0.013068755481761452, 0.001148287810260995, 0.0035867938692207835, 0.004471074967074662, 0.005059790928478192, 0.002204433187104504, 0.0011562270618438175, 0.010863432762525882, 0.0044651536264228575, 0.0010647253055616757, 0.0010930587244887269, 0.009519681544471692, 0.010139827261363199, 0.01118549041314

##### Create n samples from the dataset and perform decision tree regression on each sample and take the average of the score of each samples

In [51]:
n=10
sco=[]
predlist=[]
corlist=[]
predlen=[]
plen=[]
predlen2=[]
def make_equal_list(predlen,predlist):
    maxlen=max(predlen)
    
    for i in predlen:
        predlen2.append(maxlen-i)
    for i in range(len(predlist)):
        l=[0]*predlen2[i]
        l=np.array(l)
        predlist[i]= np.concatenate((predlist[i],l))
    return predlist
while(n):
    col=7
    row=random.randint(0,457897)
    col_list=[]
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    #print("x: ",X.iloc[10])
    #print(X)
    Y=data.iloc[0:row,34]
    #print(type(Y))
    #print("y: ",Y.iloc[10])
    reg=DecisionTreeRegressor(max_depth=3,min_samples_split=3)
    reg.fit(X,Y)
    y_pred= reg.predict(X)
    #print("ypred",y_pred)
    predlist.append(y_pred)
    predlen.append(len(y_pred))
    #print(type(y_pred))
    #y_pred= np.array(y_pred)[indices.astype(int)]
    #l=[0,0,0,0]
    #l=np.array(l)
    #y_pred=np.concatenate((y_pred,l))
    #plen.append(len(y_pred))
    
    pred= pd.Series(data=y_pred)
    corre= Y.corr(pred)
    #print("corre",corre)
    corlist.append(corre)
    r_sq = reg.score(X,Y)
    sco.append(r_sq)
    n=n-1
predlist = make_equal_list(predlen,predlist)
#print(sco)
print(corlist)
print(predlist)
predavg=[]
#print(len(predlist[0]))
for i in range(len(predlist[0])):
    j=0
    summ=0
    while(j<len(predlist)):
        summ+=predlist[j][i]
        j+=1
    avg=summ/len(predlist)
    #print(avg)
    predavg.append(avg)
#print(predavg)
pred= pd.Series(data=predavg)
corre= Y.corr(pred)
print("correlation",corre)
ybar = np.mean(Y)
sst=0
ssr=0
for i in Y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print("cod",cod)
s=0
for i in sco:
    s+=i
print("average score ",s/(len(sco)))

[0.25597736891176015, 0.2630864264529495, 0.2657893952950307, 0.26298286806036375, 0.260575454144381, 0.267974891748497, 0.26905000831726233, 0.26638258805160314, 0.2591165491515043, 0.2648470645133378]
[array([7.73184951, 7.73184951, 7.73184951, ..., 0.        , 0.        ,
       0.        ]), array([ 0.30917735,  0.30917735, -1.16478433, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.24288434,  0.24288434, -1.27539776, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.31444799,  0.31444799, -1.15070259, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.35405048,  0.35405048, -1.15596728, ...,  0.89050844,
        0.89050844,  0.89050844]), array([ 0.25670492,  0.25670492, -1.30547502, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.26104589,  0.26104589, -1.23606574, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.22714496,  0.22714496, -1.29065923, ...,  0.        ,
        0.        ,  0.        ]), array([ 0.

In [ ]:
n=10
sco=[]
dcount={}
davg={}
for i in feature_cols_JUN:
    dcount.update({i : 0})
    davg.update({i : 0})
while(n):
    col=random.randint(1,7)
    row=random.randint(0,457897)
    col_list=[]
    if(n%2==0):
        feature_cols_JUN.reverse()
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=DecisionTreeRegression()
    reg.fit(X,Y)
    Coef = pd.Series(reg.coef_,index=col_list)
    for index, value in Coef.items():
        dcount[index]+=1
        davg[index]+=value
    n=n-1
print(dcount)
print(davg)
for key,val in davg.items():
    temp= dcount[key]
    davg[key]=val/temp
print(davg)

##### Performance summary of Decision Tree Regressor for different parameter values
max_depth=2,min_samples_split=3
- [0.05539390572681868, 0.05611628638110211, 0.05647667255575728, 0.05635894442651335, 0.05499305814888733, 0.058134613810462255, 0.05544260229324872, 0.056712263695307345, 0.05643115533275012, 0.05672075943471744]
- score= 0.05627802618055646

max_depth=3,min_samples_split=3
- [0.06736145830429319, 0.06868438578135794, 0.07188646538389387, 0.06718991192169377, 0.07218417446677816, 0.0671390940546105, 0.06806889379512648, 0.07240972456504313, 0.06839622189149486, 0.06895803321852612]
- score = 0.0692278363382818

max_depth=3,min_samples_split=4
- [0.06710448586813778, 0.0689795919301679, 0.06903250067390698, 0.06872724074905046, 0.07005834520965748, 0.06900868471627186, 0.0684741484630892, 0.07066631577390692, 0.06912467759929974, 0.06666087859054592]
- score= 0.06878368695740342

max_depth=4,min_samples_split=4
- [0.08003270038793886, 0.08316556325322932, 0.08396232324892916, 0.08362677353960302, 0.07972533664277182, 0.07806711126556509, 0.07946676995310065, 0.07936497282987998, 0.07898571584915004, 0.08235950006081738]
- score= 0.08087567670309853

max_depth=5,min_samples_split=4
- [0.08428074816490283, 0.08726894266168261, 0.08732051284946218, 0.08992368723799504, 0.08471919823753238, 0.08710982811954793, 0.09264713968952565, 0.0876982244711485, 0.08592083758283287, 0.08762102235016545]
- 0.08745101413647954

##### Decision tree Regression by splitting the data into test and train

In [8]:
n=10
sco_train=[]
sco_test=[]
testpred=[]
trainpred=[]
predpred=[]
while(n):
    col=random.randint(2,7)
    row=random.randint(0,457897)
    col_list=[]
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    x_train,x_test,y_train,y_test= train_test_split(X,Y,test_size=0.3,random_state=1)
    reg=DecisionTreeRegressor(max_depth=5,min_samples_split=2)
    reg.fit(x_train,y_train)
    r_sq = reg.score(x_train,y_train)
    sco_train.append(r_sq)
    r_sq2 = reg.score(x_test,y_test)
    sco_test.append(r_sq2)
    y_pred = reg.predict(X)
    #y_pred2 = reg.predict(x_train)
    #testpred.append(y_pred)
    #trainpred.append(y_pred2)
    predpred.append(y_pred)
    n=n-1
s=0
for i in sco_train:
    s+=i
print("train: ",s/(len(sco_train)))
s=0
for i in sco_test:
    s+=i
print("test: ",s/(len(sco_test)))
#print(len(testpred[0]))
#print(len(trainpred[0]))
#print(len(testpred[1]))
#print(len(trainpred[1]))
#print(len(testpred[2]))
print(len(predpred[0]))
print(len(predpred[1]))

train:  0.08803353044594533
test:  0.08223355091441888
289873
429605


max_depth=2,min_samples_split=2
- train:  0.05106186305124892
- test:  0.04930945768028309
max_depth=2,min_samples_split=3
- train:  0.05468168735251509
- test:  0.05378289796203556

max_depth=2,min_samples_split=4
- train:  0.047735186724343556
- test:  0.046667267000722004

max_depth=3,min_samples_split=3
- train:  0.0668297180513924
- test:  0.0652018954046211
max_depth=4,min_samples_split=3
- train:  0.077166777511499
- test:  0.07335019819556957



##### Calculating average weights by taking sub-samples from the dataset and then predicting accordingly

In [12]:
n=10
sco=[]
dcount={}
davg={}
for i in feature_cols_JUN:
    dcount.update({i : 0})
    davg.update({i : 0})
while(n):
    col=random.randint(1,7)
    row=random.randint(0,457897)
    col_list=[]
    if(n%2==0):
        feature_cols_JUN.reverse()
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    Coef = pd.Series(reg.coef_,index=col_list)
    for index, value in Coef.items():
        dcount[index]+=1
        davg[index]+=value
    n=n-1
print(dcount)
print(davg)
for key,val in davg.items():
    temp= dcount[key]
    davg[key]=val/temp
print(davg)

{'trade': 4, 'event': 5, 'ob': 5, 'spreadreturn': 6, 'niftyidevent': 7, 'niftyidob': 7, 'niftyidindex': 6}
{'trade': 0.500308121042817, 'event': 0.7460848717666155, 'ob': 0.7277844407711833, 'spreadreturn': 0.6317658048686676, 'niftyidevent': 0.5189069901499902, 'niftyidob': -0.12066254008837482, 'niftyidindex': 0.1251180269490195}
{'trade': 0.12507703026070424, 'event': 0.1492169743533231, 'ob': 0.14555688815423667, 'spreadreturn': 0.1052943008114446, 'niftyidevent': 0.07412957002142717, 'niftyidob': -0.017237505726910688, 'niftyidindex': 0.02085300449150325}


##### Predicting return800 values on the basis of new averae weights obtained above

In [13]:
x=data[feature_cols_JUN]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]
        s+=m
    pred.append(s)

#print(s)
pred= pd.Series(data=pred)
corre= y.corr(pred)
print(corre)

0.05577087748891574


##### Calculating correlation manually

In [14]:
l1=[]
l2=[]
for i in range(len(y)):
    if(y[i]>0):
        l1.append(1)
    elif(y[i]<0):
        l1.append(-1)
    else:
        l1.append(0)
        
for i in range(len(pred)):
    if(pred[i]>0):
        l2.append(1)
    elif(pred[i]<0):
        l2.append(-1)
    else:
        l2.append(0)

print(len(l1))
print(len(l2))
pred= pd.Series(data=pred)
pred1 = pd.Series(data=l1)
pred2 = pd.Series(data=l2)
#corr1 = y_model_1.corr(pred1)
#corr2 = y_model_2.corr(pred2)
co= pred1.corr(pred2)
print(co)

457897
457897
0.054671464209420255


##### calculating Coefficient of Determination(score) manually

In [15]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print(cod)

8033500.013158645
44069.21166962489
0.005485680163993375


#### After including the intercept

In [16]:
n=10
p=n
sco=[]
dcount={}
davg={}
intersum=0
for i in feature_cols_JUN:
    dcount.update({i : 0})
    davg.update({i : 0})
while(n):
    col=random.randint(1,7)
    row=random.randint(0,457897)
    col_list=[]
    if(n%2==0):
        feature_cols_JUN.reverse()
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    Coef = pd.Series(reg.coef_,index=col_list)
    #print(Coef)
    #print()
    inc = reg.intercept_
    intersum+=inc
    for index, value in Coef.items():
        dcount[index]+=1
        davg[index]+=value
    n=n-1
print(dcount)
print(davg)
for key,val in davg.items():
    temp= dcount[key]
    davg[key]=val/temp
intavg=intersum/p
print(intavg)
print(davg)

{'niftyidindex': 4, 'niftyidob': 5, 'niftyidevent': 7, 'spreadreturn': 8, 'ob': 7, 'event': 8, 'trade': 7}
{'niftyidindex': 0.10976240161462388, 'niftyidob': -0.09699148467778886, 'niftyidevent': 0.44755817317136276, 'spreadreturn': 0.540000860286138, 'ob': 1.3943514807959831, 'event': 0.978849033309045, 'trade': 0.7886622537376611}
0.018007871227688065
{'niftyidindex': 0.02744060040365597, 'niftyidob': -0.019398296935557773, 'niftyidevent': 0.06393688188162325, 'spreadreturn': 0.06750010753576725, 'ob': 0.19919306868514045, 'event': 0.12235612916363063, 'trade': 0.1126660362482373}


In [17]:
x=data[feature_cols_JUN]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]
        s+=m
    s+=intavg
    pred.append(s)
pred= pd.Series(data=pred)
corre= y.corr(pred)
print(corre)

0.05532671994239343


In [18]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print(cod)

8033500.013158645
39129.918147886645
0.004870843104972048


###### Instead of taking the average weights , here the average weights are multiplied by some random constants to observe the changes in predicted values

In [54]:
w=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
print(davg)
k=0
for i,j in davg.items():
    ran=random.randint(0,9)
    davg[i]=(davg[i]*w[k])/sum(w)
    k+=1
print(davg)

{'niftyidindex': 0.00050551785244986, 'niftyidob': -4.463864905530456e-05, 'niftyidevent': 0.0002712238479321227, 'spreadreturn': 0.00022042452963148403, 'ob': 0.00038695571915674156, 'event': 7.623560793607085e-05, 'trade': 0.0006383418847845569}
{'niftyidindex': 9.191233680906547e-06, 'niftyidob': -1.6232236020110748e-06, 'niftyidevent': 1.4794028069024875e-05, 'spreadreturn': 1.603087488228975e-05, 'ob': 3.517779265061287e-05, 'event': 8.316611774844093e-06, 'trade': 8.12435126089436e-05}


In [19]:
x=data[feature_cols_JUN]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]
        s+=m
    s+=intavg
    pred.append(s)
pred= pd.Series(data=pred)
corre= y.corr(pred)
print(corre)

0.05532671994239343


In [20]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print(cod)

8033500.013158645
39129.918147886645
0.004870843104972048


##### It can be clearly concluded by observing the new score that the constant random weights are not helping in better prediction of the model

##### Using RMSE
- Dividing each weight by rmse values of models and calculating the score to observe changes in the predicted value

In [21]:
n=10
p=n
sco=[]
dcount={}
davg={}
ms=[]
intersum=0
for i in feature_cols_JUN:
    dcount.update({i : 0})
    davg.update({i : 0})
while(n):
    col=random.randint(1,7)
    row=random.randint(0,457897)
    col_list=[]
    if(n%2==0):
        feature_cols_JUN.reverse()
    for i in range(len(feature_cols_JUN)):
        if(i<=col):
            col_list.append(feature_cols_JUN[i])
    XX=data[col_list]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    Coef = pd.Series(reg.coef_,index=col_list)
    #print(Coef)
    #print()
    inc = reg.intercept_
    intersum+=inc
    for index, value in Coef.items():
        dcount[index]+=1
        davg[index]+=value
    from sklearn.metrics import r2_score,mean_squared_error
    y_pred = reg.predict(X)
    mse=mean_squared_error(Y,y_pred)
    rmse= np.sqrt(mse)
    ms.append(rmse)
    n=n-1
print(dcount)
print(davg)
for key,val in davg.items():
    temp= dcount[key]
    davg[key]=val/temp
intavg=intersum/p
print(intavg)
print(davg)
print(ms)

{'trade': 4, 'event': 7, 'ob': 6, 'spreadreturn': 6, 'niftyidevent': 9, 'niftyidob': 8, 'niftyidindex': 8}
{'trade': 0.4704630577033942, 'event': 0.9054206369791131, 'ob': 0.994044100168474, 'spreadreturn': 0.33842938863228567, 'niftyidevent': 0.6045833296936566, 'niftyidob': -0.16439179550584587, 'niftyidindex': 0.15845355315133292}
0.019952099199730282
{'trade': 0.11761576442584855, 'event': 0.12934580528273046, 'ob': 0.16567401669474566, 'spreadreturn': 0.05640489810538094, 'niftyidevent': 0.0671759255215174, 'niftyidob': -0.020548974438230734, 'niftyidindex': 0.019806694143916614}
[4.147175742668686, 3.6212427297212817, 3.629416790473557, 4.3305334349688165, 3.8604166814381613, 3.6410263818038486, 4.181350454687284, 4.34457237881039, 4.2856529451599155, 4.231834500186007]


In [26]:
x=data[feature_cols_JUN]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]*(1/ms[3])
        s+=m
    s+=intavg
    pred.append(s)
pred= pd.Series(data=pred)
corre= y.corr(pred)
print("correlation: ",corre)

correlation:  0.05576328348208124


In [27]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print("cod: ",cod)

8033500.013158645
2006.3295419993499
cod:  0.00024974538354553295


##### Summary for different rmse values

ms[0]
- corre = 0.05488660668394191
- cod= 0.0004482884204472265

correlation:  0.054886606683942185
cod:  0.00032474037640212823

ms[1]
- correlation:  0.055763283482081344
- cod:  0.00034833375999771357

ms[2]
- correlation:  0.055763283482081386
- cod:  0.0003468553250985894

ms[3]
- correlation:  0.05576328348208124
- cod:  0.00024974538354553295


##### Using score values of the model
- Multiplying each weight by r_score values of models and calculating the score to observe changes in the predicted value

In [28]:
n=10
p=n
sco=[]
dcount={}
davg={}
intersum=0
rs=[]
for i in feature_cols_JUN:
    dcount.update({i : 0})
    davg.update({i : 0})
while(n):
    #col=random.randint(1,7)
    row=random.randint(0,457897)
    XX=data[feature_cols_JUN]
    X=XX.iloc[0:row,:]
    Y=data.iloc[0:row,34]
    reg=LinearRegression()
    reg.fit(X,Y)
    Coef = pd.Series(reg.coef_,index=feature_cols_JUN)
    #print(Coef)
    #print()
    inc = reg.intercept_
    intersum+=inc
    for index, value in Coef.items():
        dcount[index]+=1
        davg[index]+=value
    from sklearn.metrics import r2_score,mean_squared_error
    y_pred = reg.predict(X)
    rsq=r2_score(Y,y_pred)
    rs.append(rsq)
    n=n-1
print(dcount)
print(davg)
for key,val in davg.items():
    temp= dcount[key]
    davg[key]=val/temp
intavg=intersum/p
print(intavg)
print(davg)
print(rs)

{'niftyidindex': 10, 'niftyidob': 10, 'niftyidevent': 10, 'spreadreturn': 10, 'ob': 10, 'event': 10, 'trade': 10}
{'niftyidindex': 0.344347393072848, 'niftyidob': -0.09794469506954775, 'niftyidevent': 0.49140115873680257, 'spreadreturn': 0.6227252650815502, 'ob': 2.2474386878058574, 'event': 1.0678039265543844, 'trade': 1.3863386569737537}
0.04931718293531808
{'niftyidindex': 0.0344347393072848, 'niftyidob': -0.009794469506954775, 'niftyidevent': 0.04914011587368026, 'spreadreturn': 0.06227252650815503, 'ob': 0.22474386878058575, 'event': 0.10678039265543844, 'trade': 0.13863386569737537}
[0.003850647365639226, 0.011816310811142983, 0.0034499479986951354, 0.00516020014827534, 0.0032628385389318337, 0.003457910953028276, 0.010195886358191797, 0.012004087711736111, 0.010079262043979131, 0.0037535338928214035]


In [31]:
x=data[feature_cols_JUN]
y= data['return800']
pred=[]
for j in range(len(x)):
    s=0
    for i in x.columns:
        m=davg[i]*x.at[j,i]*rs[0]
        s+=m
    s+=intavg
    pred.append(s)
pred= pd.Series(data=pred)
corre= y.corr(pred)
print("correlation: ",corre)

correlation:  0.053727462656701905


In [32]:
ybar = np.mean(y)
sst=0
ssr=0
for i in y:
    ss= (i-ybar)**2
    sst+=ss
print(sst)
for i in pred:
    ss2= (i-ybar)**2
    ssr+=ss2
print(ssr)
cod= ssr/sst
print("coefficient of determinnation: ",cod)

8033500.013158645
1220.5885228404989
coefficient of determinnation:  0.0001519373275460521
